# Poker Analysis

### Preamble

In [1]:
%%time

# create list of ranks and suits
ranks = [str(n) for n in range(2, 10)] + list('TJQKA')
suits = list('♠️♥️♦️♣️')[::2]

# create string of ranks
straight_ranks_str = 'A' + ''.join(ranks)

# create list of cards
cards = [r+s for s in suits for r in ranks]

# create all combinations of 5 card hands ()
from itertools import combinations
hands = list(combinations(cards, 5))

print(f'{len(hands):,}') # check length


2,598,960
Wall time: 509 ms


### Intermediary Functions

In [2]:
def group_by(hand, by_rank=True):
    
    # check if by rank or by suit
    if by_rank:
        i = 0
    else:
        i = 1
        
    hand_ranks = []
    for card in hand:
        hand_ranks.append(card[i])

    # create dictionary with count by group
    res = dict()
    for rank in set(hand_ranks):
        res[rank] = hand_ranks.count(rank)
    
    return res

# test function
hand = ('2♠', '3♠', '4♠', 'J♠', 'J♣')
print(group_by(hand))
print(group_by(hand, by_rank=False))

{'4': 1, '2': 1, '3': 1, 'J': 2}
{'♠': 4, '♣': 1}


In [3]:
def is_straight(rank_group, n=5):
    

    # sort ranks
    ranks_str = ''.join(
        sorted(
            set(rank_group),
            key=lambda x : straight_ranks_str.index(x) # sort by index in straight_ranks
        )
    )
    
    # iterate n-lengthed substrings
    for i in range(len(ranks_str)-n+1):

        ranks_substr = ranks_str[i:i+n]

        # check if n-substring is in straight string
        if ranks_substr in straight_ranks_str:
            return True
        
    # special case of Ace at the end
    if ranks_str[-4:] + ranks_str[0] in straight_ranks_str:
        return True

    return False

# test function
hand = ('2♠', '3♠', '4♠', '6♠', 'J♣', '5♣')
rank_group = group_by(hand)
print(is_straight(rank_group))

True


In [4]:
# dictionary of hand names

hand_names = {
    
    1 : 'Straight flush',
    2 : '4 of a kind',
    3 : 'Full house',
    4 : 'Flush\t',
    5 : 'Straight',
    6 : '3 of a kind',
    7 : '2 pair\t',
    8 : 'Pair\t',
    9 : 'High card'
}

# check print alignment
for name in hand_names:
    print(hand_names[name], '\t.')

Straight flush 	.
4 of a kind 	.
Full house 	.
Flush	 	.
Straight 	.
3 of a kind 	.
2 pair	 	.
Pair	 	.
High card 	.


In [5]:
def id_hand(hand):
    
    # dictionary of ranks and counts
    rank_group = group_by(hand) 
    rank_counts = list(rank_group.values()) # just counts of ranks
    
    # dictionary of suits and counts
    suit_group = group_by(hand, by_rank=False)
    suit_counts = list(suit_group.values()) # just count of suits
    
    
    ## straight flush
    
    # create dictionary of ranks by suit
    suited_ranks = {suit : set() for suit in suit_group}
    for rank, suit in hand:
        suited_ranks[suit].add(rank)

    # for set of same-suited ranks
    for suited_rank in suited_ranks.values():

        # join ranks in a string
        if is_straight(suited_rank):
            return 1
        
    
    ## four of a kind
    k = 4
    if (k in rank_counts):
        return 2
    
    ## full house
    if 2 in rank_counts and 3 in rank_counts:
        return 3
    
    ## flush
    if 5 in suit_counts:
        return 4
    
    ## straight
    if is_straight(rank_group):
        return 5
    
    ## three of a kind
    k = 3
    if (k in rank_counts):
        return 6
    
    ## two pair
    if rank_counts.count(2) >= 2:
        return 7
    
    ## pair
    k = 2
    if (k in rank_counts):
        return 8
    
    
    ## last and least: high card
    return 9

# test
hand = ('2♠', '2♥', '2♦', '2♣', 'J♦')
hand_names[id_hand(hand)]

'4 of a kind'

### Results

In [6]:
%%time

# count all hands
# run-time ~ 1.25 min

hand_counts = dict()
hands_len = len(hands)
for i, hand in enumerate(hands):
    
    if i % 100_000 == 0:
        print(f'{i/hands_len:.2%} \t ({i:,} of {hands_len:,})')
    
    hand_type = id_hand(hand)
    
    if hand_type not in hand_counts:
        hand_counts[hand_type] = 1
    else:
        hand_counts[hand_type] += 1
    
print('Complete!')

0.00% 	 (0 of 2,598,960)
3.85% 	 (100,000 of 2,598,960)
7.70% 	 (200,000 of 2,598,960)
11.54% 	 (300,000 of 2,598,960)
15.39% 	 (400,000 of 2,598,960)
19.24% 	 (500,000 of 2,598,960)
23.09% 	 (600,000 of 2,598,960)
26.93% 	 (700,000 of 2,598,960)
30.78% 	 (800,000 of 2,598,960)
34.63% 	 (900,000 of 2,598,960)
38.48% 	 (1,000,000 of 2,598,960)
42.32% 	 (1,100,000 of 2,598,960)
46.17% 	 (1,200,000 of 2,598,960)
50.02% 	 (1,300,000 of 2,598,960)
53.87% 	 (1,400,000 of 2,598,960)
57.72% 	 (1,500,000 of 2,598,960)
61.56% 	 (1,600,000 of 2,598,960)
65.41% 	 (1,700,000 of 2,598,960)
69.26% 	 (1,800,000 of 2,598,960)
73.11% 	 (1,900,000 of 2,598,960)
76.95% 	 (2,000,000 of 2,598,960)
80.80% 	 (2,100,000 of 2,598,960)
84.65% 	 (2,200,000 of 2,598,960)
88.50% 	 (2,300,000 of 2,598,960)
92.34% 	 (2,400,000 of 2,598,960)
96.19% 	 (2,500,000 of 2,598,960)
Complete!
Wall time: 1min 44s


In [7]:
for num in sorted(hand_counts):
    
    count = hand_counts[num]
    
    print(f'{hand_names[num]}\t: \t {count / hands_len:%} \t ({count} in {hands_len})')

Straight flush	: 	 0.001539% 	 (40 / 2598960)
4 of a kind	: 	 0.024010% 	 (624 / 2598960)
Full house	: 	 0.144058% 	 (3744 / 2598960)
Flush		: 	 0.196540% 	 (5108 / 2598960)
Straight	: 	 0.392465% 	 (10200 / 2598960)
3 of a kind	: 	 2.112845% 	 (54912 / 2598960)
2 pair		: 	 4.753902% 	 (123552 / 2598960)
Pair		: 	 42.256903% 	 (1098240 / 2598960)
High card	: 	 50.117739% 	 (1302540 / 2598960)


### Seven-card hands (Texas Hold'em)

In [8]:
%%time
seven_card_hands = list(combinations(cards, 7))

print(f'{len(seven_card_hands):,}') # check length

133,784,560
Wall time: 37.4 s


In [12]:
import time
report_time = lambda t : f'{t // 3600:.0f}h {(t // 60) % 60:.0f}m {t % 60:.0f}s'


def count_hands(hands, update=100_000):

    # count all hands
    t0 = time.time()

    hand_counts = dict()
    hands_len = len(hands)

    for i, hand in enumerate(hands):

        if i % update == 0:

            print(f'{i/hands_len:.2%} \t ({i:,} of {hands_len:,})')

            dt = time.time() - t0 # change
            eta = dt * (hands_len-i)/ (i+1) # remaining time
            print(f'Elapsed:', report_time(dt))
            print(f'ETA:', report_time(eta))
            print('-'*15)

        hand_type = id_hand(hand)

        if hand_type not in hand_counts:
            hand_counts[hand_type] = 1
        else:
            hand_counts[hand_type] += 1

    print('Complete!')

    return hand_counts

In [13]:
# ETA > 1 hr

count7 = count_hands(seven_card_hands, update=500_000)

0.00% 	 (0 of 133,784,560)
Elapsed: 0h 0m 0s
ETA: 38h 39m 25s
---------------
0.37% 	 (500,000 of 133,784,560)
Elapsed: 0h 0m 21s
ETA: 1h 31m 13s
---------------
0.75% 	 (1,000,000 of 133,784,560)
Elapsed: 0h 0m 42s
ETA: 1h 32m 7s
---------------
1.12% 	 (1,500,000 of 133,784,560)
Elapsed: 0h 1m 3s
ETA: 1h 33m 12s
---------------
1.49% 	 (2,000,000 of 133,784,560)
Elapsed: 0h 1m 25s
ETA: 1h 33m 33s
---------------
1.87% 	 (2,500,000 of 133,784,560)
Elapsed: 0h 1m 46s
ETA: 1h 32m 39s
---------------
2.24% 	 (3,000,000 of 133,784,560)
Elapsed: 0h 2m 11s
ETA: 1h 35m 28s
---------------
2.62% 	 (3,500,000 of 133,784,560)
Elapsed: 0h 2m 37s
ETA: 1h 37m 16s
---------------
2.99% 	 (4,000,000 of 133,784,560)
Elapsed: 0h 3m 1s
ETA: 1h 37m 42s
---------------
3.36% 	 (4,500,000 of 133,784,560)
Elapsed: 0h 3m 22s
ETA: 1h 36m 33s
---------------
3.74% 	 (5,000,000 of 133,784,560)
Elapsed: 0h 3m 45s
ETA: 1h 36m 39s
---------------
4.11% 	 (5,500,000 of 133,784,560)
Elapsed: 0h 4m 9s
ETA: 1h 36m 50

KeyboardInterrupt: 